<h3>Applied Data Science Capstone - Week 3</h3>
<h3>Peer Graded Assignment Part 2</h3>

##### ---------------------------------------------------------Part 1 Begin-------------------------------------------------------------------------

### Importing necessary libraries

In [61]:
import urllib.request 
import pandas as pd
from bs4 import BeautifulSoup

### Using Beautiful Soup to retrieve html containing postal codes of Canada

In [62]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
soup = BeautifulSoup(urllib.request.urlopen(url))
# soup  #Uncomment to print complete retrieved html form the above url

### Using beautiful soup inbuilt function to retrive table from the url html data

In [63]:
table = soup.find_all('table')
df = pd.read_html(str(table))[0] # save table directly to a dataframe

In [64]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [65]:
df.shape ## Shape before cleaning

(180, 3)

### Cleaning data
<p> Removing rows that contain Borough value as 'Not Assigned'</p>

In [66]:
df = df[df.Borough != 'Not assigned']

In [67]:
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


 #### Notice first two rows that had Borough Not Assigned have been removed

### Display no. of rows in the dataframe using .shape method

In [68]:
df.shape #Shape after cleaning.

(103, 3)

##### ---------------------------------------------------------Part 1 End-------------------------------------------------------------------------


##### ---------------------------------------------------------Part 2 Begin-------------------------------------------------------------------------

### Using pandas to read coordinates-data to a dataframe

In [69]:
coords = pd.read_csv('http://cocl.us/Geospatial_data')

In [70]:
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### We now have two dataframes - df and coords
>   df - Postal Code | Borough | Neighborhood
>   
coords - Postal Code | Latitude | Longitude

   *Joining these to dataframes on Postal Code column will give us the desired dataframe which will contain -*
>   #### Postal Code | Borough | Neighborhood | Latitude | Longitude

In [71]:
join_df = pd.merge(df,coords,on='Postal Code')

In [72]:
join_df.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


##### ---------------------------------------------------------Part 2 End-------------------------------------------------------------------------

##### ---------------------------------------------------------Part 3 Begin-------------------------------------------------------------------------

### Importing necessary libraries

In [73]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /Users/mb/anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::altair-4.1.0-py_1
  branca             conda-forge/noarch::branca-0.4.1-py_0
  folium             conda-forge/noarc

### Using geopy library to find coordinates of Toronto

In [102]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="user_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


### Displaying map with all the Boroughs in Blue and Toronto in Red

In [103]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10) # generate map centred around the Toronto


for lat, lng, label in zip(join_df['Latitude'], join_df['Longitude'], join_df['Borough']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(toronto_map)

folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.4
).add_to(toronto_map)



toronto_map